This is the software companion to the paper [Computing Galois groups of Ehrhart polynomials in OSCAR](https://www.mat.univie.ac.at/~slc/wpapers/FPSAC2022/87.pdf) by [Claus Fieker](https://www.mathematik.uni-kl.de/~fieker/en/publications.html),
[Tommy Hofmann](https://www.thofma.com/) and
[Michael Joswig](https://page.math.tu-berlin.de/~joswig/). 

This notebook was originally written for version 0.8.2 of OSCAR (18 Mar 2022), but now we are using v0.14.0 (13 Jan 2024).  To run this code [install Julia and OSCAR](https://www.oscar-system.org/install/), start the Julia interpreter.

In [1]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  Version 1.0.2


The Ehrhart polynomial of the cube factors into linear factors, whence the Galois group is trivial.  Here we show it for the 3-cube, but this generalizes to arbitrary dimensions.

Due to Julia's just-in-time compilation the command below will take a while upon first computation.

In [2]:
C = cube(2)

LC = ehrhart_polynomial(C)
@show factor(LC)

galois_group(LC)

factor(LC) = 1 * (2*x + 1)^2


(Permutation group of degree 2 and order 1, Galois context for 2*x + 1 and prime 3)

The Fano simplices exist in all dimensions, they have the origin as the unique interior lattice point.

In [3]:
S = fano_simplex(14)

Polytope in ambient dimension 14

The normal toric variety associated with the face fan is smooth; in fact, it is a projective space.

In [4]:
is_smooth(normal_toric_variety(face_fan(S)))

true

Here we are interested in the Galois group of the Ehrhart polynomial.  This is our first non-trivial example.

Timings on 11th Gen Intel Core i7-1165G7 (6.8.9-3-MANJARO).

In [5]:
@time G = galois_group(ehrhart_polynomial(S))

  4.261074 seconds (7.96 M allocations: 513.861 MiB, 2.03% gc time, 78.86% compilation time)


(Permutation group of degree 14 and order 645120, Galois context for 15*x^14 + 105*x^13 + 24115*x^12 + 143325*x^11 + 7724717*x^10 + 37312275*x^9 + 725938785*x^8 + 2681453775*x^7 + 21964438496*x^6 + 56663366760*x^5 + 201186840400*x^4 + 310989260400*x^3 + 429952217472*x^2 + 283465647360*x + 87178291200 and prime 59)

The Ehrhart polynomials of the Fano simplices are known.  Here we give a closed form description. 

In [6]:
Qt, t = QQ["t"]

LS(d) = binomial(t+d+1,d+1) - binomial(t,d+1)

LS (generic function with 1 method)

Here is the case $d=6$, normalized to rendering the coefficients integral.

In [7]:
factorial(6) * LS(6)

7*t^6 + 21*t^5 + 385*t^4 + 735*t^3 + 2128*t^2 + 1764*t + 720

Its roots lie on the *critical line* $\operatorname{Re}(z)=-1/2$.

In [8]:
roots(QQBarField(), LS(6))

6-element Vector{QQBarFieldElem}:
 Root -0.500000 + 0.446648*im of 7x^6 + 21x^5 + 385x^4 + 735x^3 + 2128x^2 + 1764x + 720
 Root -0.500000 - 0.446648*im of 7x^6 + 21x^5 + 385x^4 + 735x^3 + 2128x^2 + 1764x + 720
 Root -0.500000 + 2.15772*im of 7x^6 + 21x^5 + 385x^4 + 735x^3 + 2128x^2 + 1764x + 720
 Root -0.500000 - 2.15772*im of 7x^6 + 21x^5 + 385x^4 + 735x^3 + 2128x^2 + 1764x + 720
 Root -0.500000 + 6.81137*im of 7x^6 + 21x^5 + 385x^4 + 735x^3 + 2128x^2 + 1764x + 720
 Root -0.500000 - 6.81137*im of 7x^6 + 21x^5 + 385x^4 + 735x^3 + 2128x^2 + 1764x + 720

In this case the structure of Galois group can be recognized.  Note that one group occurs under many names.

In [9]:
describe(galois_group(LS(6))[1])

"C2 x S4"

OSCAR provides access to the database [polyDB](https://db.polymake.org).  Among other objects it comprises the complete list of smooth Fano polytopes of dimension $\leq 8$.

Notice, however, that OSCAR defines toric varieties from lattice polytopes through normal fans by default.  Hence all "smooth" polytopes are necessarily simple.

Our goal is to provide a tally of orders of Galois groups of Ehrhart polynomials of the polars of all $6$-dimensional polytopes from that collection.

In [10]:
db = Polymake.Polydb.get_db();
collection = db["Polytopes.Lattice.SmoothReflexive"];
query = Dict("DIM"=>6);
results = Polymake.Polydb.find(collection, query);

We collect the polars of those polytopes and their Ehrhart polynomials.  While the polytopes come from the database, their polars, and the Ehrhart polynomials of the polars are actually computed, so this is a nontrivial computation.

In [11]:
PP6 = [ polarize(polyhedron(P)) for P in results ]
E6 = [ ehrhart_polynomial(P) for P in PP6 ];

size(E6)

(7622,)

Here is the final statistics, which is a multiset of orders of 7622 Galois groups.  This is the most costly computation in this notebook.

In [23]:
order(galois_group(LS(6))[1])

48

In [47]:
MSet([order(galois_group(E6[i])[1]) for i in range(1,310)])

MSet{ZZRingElem} with 310 elements:
  16 : 42
  12 : 13
  8  : 3
  48 : 252

In [48]:
galois_group(E6[311])

LoadError: Order must be a prime power

Above we had mentioned the critical line $\operatorname{Re}(z)=-1/2$.

In [13]:
function on_critical_line(L)    
    for z in roots(L, ComplexField())
        if real(z) != -1/2
            return false
        end
    end
    return true
end

on_critical_line (generic function with 1 method)

The following command duplicates what we already saw above.

In [14]:
on_critical_line(LS(6))

true

Hegedüs and Kasprzyk (2011) showed that there are exactly four types of polars of smooth Fano $6$-polytopes such that the zeros of their Ehrhart polynomials do not lie on the critical line.

Let's find them (i.e., their indices).

In [15]:
function check_critical_line(EE)
    idx = 1
    non_critical = []
    for e in EE
        if !on_critical_line(e)
            push!(non_critical, idx)
        end
        idx+=1
    end
    return non_critical
end

NC6 = check_critical_line(E6)

4-element Vector{Any}:
  721
  804
 2617
 3155

And these are their Galois groups.  The one with index 3155 is lattice isomorphic to (the polar of) a free sum of del Pezzo polytopes $\operatorname{DP}(2)\oplus\operatorname{DP}(4)$.

In [16]:
f_vector(polarize(del_pezzo_polytope(2)) * polarize(del_pezzo_polytope(4)))

6-element Vector{ZZRingElem}:
 180
 540
 630
 360
 106
 16

It turns out that this is the only (polar of a) smooth Fano polytope whose Ehrhart polynomial has a Galois group of order six; re-check the tally above.  

In [17]:
[ describe(galois_group(E6[k])[1]) for k in NC6 ]

4-element Vector{String}:
 "C2 x S4"
 "C2 x S4"
 "C2 x S4"
 "S3"

Let's look at the vertices of the polytope with that special Galois group.

In [18]:
vertices(PP6[3155])

16-element SubObjectIterator{PointVector{QQFieldElem}}:
 [0, 0, 1, 0, 0, -1]
 [0, 0, -1, 0, 0, 1]
 [0, 0, 0, -1, 0, 0]
 [0, 0, 1, 0, 0, 0]
 [0, 0, 0, 0, 1, 0]
 [-1, -1, 0, 1, 1, 0]
 [0, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, -1, 0]
 [0, -1, 0, 0, 0, 0]
 [1, 0, 0, 0, 0, 0]
 [-1, 0, 0, 0, 0, 0]
 [0, 0, -1, 0, 0, 0]
 [0, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1]
 [1, 1, 0, -1, -1, 0]
 [0, 0, 0, 0, 0, -1]